<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/PassaggichiaveMKD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Fase 1: La Scansione Originale (Il Mondo Reale in Pixel)**

*   **Immagine di Riferimento:** `T0_90_1A_ingresso.jpg` (e le altre scansioni complete).
*   **Scala Teorica (DPI):** 600 DPI (dots per inch).
    *   Conversione: 1 pollice = 25.4 mm
    *   **Scala Originale (px/mm):** `600 / 25.4 = **23.62 pixel per mm**`.
*   **Dimensione Fisica Riferimento:** Diametro nominale del foro `D_nom = **6.0 mm**`.
*   **Dimensione in Pixel (Teorica):** Sulla scansione originale, un foro da 6 mm dovrebbe avere un diametro di `6.0 mm * 23.62 px/mm ≈ **142 pixel**`.

#### **Fase 2: Dal Rilevamento YOLO al Ritaglio della Patch**

*   **Script di Riferimento:** `yolorilevazionefori.ipynb` e gli script successivi di ordinamento e crop.
*   **Processo:**
    1.  YOLO rileva i fori sulla scansione intera (`imgsz=1280` o `(H,W)` nativa).
    2.  Le coordinate delle bounding box vengono salvate e ordinate con K-Means (l'ordinamento bustrofedico).
    3.  Uno script di crop prende il centro `(cx, cy)` di ogni foro rilevato.
    4.  Viene ritagliata una patch quadrata attorno a quel centro. Hai usato `HS = 350` (patch 700x700) e `HS = 256` (patch 512x512).
    5.  **IL PASSAGGIO CRUCIALE:** In uno degli script, c'è `patch = cv2.resize(patch, (TARGET_SIZE, TARGET_SIZE), ...)` dove `TARGET_SIZE` è `512`.

#### **Fase 3: La Patch Radiografica (L'Input per la UNet)**

*   **Immagine di Riferimento:** L'immagine della patch singola che mi hai mostrato.
*   **Dimensione:** **512x512 pixel**.
*   **Il Problema:** Questa immagine è il risultato del `resize` della Fase 2. La sua scala in `pixel/mm` **NON è più 23.62**. È una nuova scala che dobbiamo calcolare.

#### **Fase 4: La Maschera Predetta (L'Output della UNet)**

*   **Immagine di Riferimento:** L'immagine della maschera in bianco e nero che mi hai mostrato.
*   **Dimensione:** **512x512 pixel** (la stessa della patch di input).
*   **Contenuto:** I valori dei pixel rappresentano le classi (es. 0=Sfondo, 1=Foro, 2=Danno).
*   **Obiettivo:** Misurare l'area e il DMAX del danno (pixel di classe 2) su questa maschera e convertirli in `mm²` e `mm` usando la scala corretta.
